# ProWave - WaveNet-based Protein Generation

Authors: Hans Jakob Damsgaard & Lucas Balling

02456 Deep Learning project: ProGen

## Initialization

Run the commmand below if you have not yet installed the [TAPE project](https://github.com/songlab-cal/tape).

In [ ]:
#!pip install tape_proteins

#### Importing needed packages

In [ ]:
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import tape

#### Importing the data

We were unable to make the data download script, `download_data.sh`, run from Jupyter, so instead we ran it manually and simply placed the resulting files in the right folder for TAPE to find them. We import all the data in the LMDB format as it is most easily worked with in Python.

In [ ]:
from tape.datasets import LanguageModelingDataset
ModelPath = './Pretrained/'
#data_path = '/Users/lucasballing/Desktop/DeepLearningProject/prowave-main/data/pfam/'
data_path = 'E:/Pfam/data/'
train_data   = LanguageModelingDataset(data_path, 'train')
valid_data   = LanguageModelingDataset(data_path, 'valid')
holdout_data = LanguageModelingDataset(data_path, 'holdout')

#### Understanding data features

To get a good understanding of the data provided in the imported dataset, we provide plots of certain features and their ranges. Data is already split into the three required subsets; train, validation, and holdout by TAPE, so it is also interesting to understand this split.

In [ ]:
# Split sizes
print(f'Training data has shape ({len(train_data)}, {len(train_data[0])})')
print(f'Validation data has shape ({len(valid_data)}, {len(valid_data[0])})')
print(f'Holdout data has shape ({len(holdout_data)}, {len(holdout_data[0])})')

# Original data columns
from tape.datasets import LMDBDataset
lmdb_train = LMDBDataset(data_path+'pfam/pfam_train.lmdb')
print(f'File data entries look like this: {lmdb_train[0]}')
del lmdb_train

# Data columns - all subsets are taken from the same overall dataset, so the columns are the same
# From combining information from LMDBDataset and LanguageModelingDataset, we know the columns are
# - IUPAC-encoded protein string
# - Input mask (for masked-token prediction)
# - Protein clan
# - Protein family
# The protein ID (i.e., its number within its clan and family) is not included
print(f'Encoded data entries look like this: {train_data[0]}')
del train_data
del valid_data
del holdout_data

In [ ]:
#from utils import setify

# Fetch results from all splits
#results = setify([train_data, valid_data, holdout_data])

# Clans in splits
#clans = results[0][0]
#print(f'Unique clans in training data {len(clans[0])}')
#print(f'Unique clans in validation data {len(clans[1])}')
#print(f'Unique clans in holdout data {len(clans[2])}')

# Families in splits
#families = results[0][1]
#print(f'Unique families in training data {len(families[0])}')
#print(f'Unique families in validation data {len(families[1])}')
#print(f'Unique families in holdout data {len(families[2])}')

# PRINTS:
# Unique clans in training data 623
# Unique clans in validation data 623
# Unique clans in holdout data 8
# Unique families in training data 17737
# Unique families in validation data 15974
# Unique families in holdout data 28

# Idea and Initial Implementation
We intend to follow the ideas presented in the ProGen paper relatively closely. That is, our network will be trained on conditioned aminoacid sequences with the only two available conditioning tags being the clan and family IDs. To enable this, our network's input will be as shown on the figure below.

<img src="../Training.png" width="500"/>

So, like ProGen, our input $x=[c;a]$ is a sequence of encoded conditioning tags $c=(c_0,...,c_n)$ (in this case just two), and a starting sequence $a=(a_0, ..., a_n)$ of aminoacids for starting the sequence generation. The sequence is fed one character at a time to the network, which accumulates state before starting generation. We intend to let the network run free either until it generates an end character or until the generated sequence has length 500.


## One-hot encoding
One way to represent a fixed amount of words is through one-hot encoding. We intend to use one-hot encoding for both aminoacids as well as conditioning tags meaning that the input characters to our network become rather large. This may seem inefficient due to the great sparsity in the vectors produced, but we intend to limit this sparsity by decreasing the number of clans and families considered. This will allow us to generate proteins only for a subset of the available clans and families but at a much shorter required training and evaluation time. 

An example of a one-hot encoding is shown below:

| Amionacid    | one-hot encoded vector   |
| ------------- |--------------------------|
| pad   | $= [0, 0, 0, \ldots, 0]$ |
| CLS   | $= [0, 1, 0, \ldots, 0]$ |
| SEP   | $= [0, 0, 1, \ldots, 0]$ | 
| Ala = A  | $= [0, 0, 0, 1, 0, \ldots, 0]$ |
| Asx = B  | $= [0, 0, 0, 0, 1,\ldots, 0]$ |
| ... | ... |

This notebook will as tape use the BERT sequence  [CLS] X [SEP] - Start and stop sequences.

Although the implementation from earlier in the course, which is used below, works well, we have decided to instead use Pytorch's own Embedding module which allows us to skip manually encoding vectors.

For more information about the one hot incoding see the TAPE: https://github.com/songlab-cal/tape/blob/master/tape/tokenizers.py

In [ ]:
from utils import one_hot_encode, one_hot_encode_sequence

# Testing the implementation
VOCAB_SIZE = 30
#test_word = one_hot_encode(1,VOCAB_SIZE)
#print(f'Our one-hot encoding of \'1\' has shape {test_word.shape}.')
#print(test_word)

#test_sentence = one_hot_encode_sequence(holdout_data[1][0], VOCAB_SIZE)
#print(f'Our one-hot encoding of \'{holdout_data[1][0]}\' has shape {test_sentence.shape}.')
#print(test_sentence)

## Reducing input size
To reduce training time and problem size, we intend to extract 10 clans and their families from the training set and use those for training the network. Data is stored in sorted order in the training data set meaning that we can simply store elements until we see the 11th clan ID at which we can stop looking through the dataset. This is preferable over having to look through the entire dataset (as we did to gather plot data earlier), as such an operation takes a long time due to the mere size of the dataset.

As part of this operation, we also throw away the input masks that are included in the original dataset and attempt to limit data size by using `int8` instead of the original `int64` format for encoded sequences.

In [ ]:
CLANS = 10
FAMILIES = 100
LENGTH = 512
LENGTH_OUT = 514

In [ ]:
# Reduce data size by picking only 10 first clans
#from utils import get_data
#for d, n in zip([train_data, valid_data, holdout_data], ['train', 'valid', 'holdout']):
#    dataset = get_data(d, CLANS)
#    with open(data_path+n+'_red.pkl', 'wb') as f:
#        pkl.dump(dataset, f)
#    del dataset

In [ ]:
# Reduce data size by picking only the 100 first families
#for i, o in zip(['train_red.pkl', 'valid_red.pkl', 'holdout_red.pkl'], ['train_red2.pkl', 'valid_red2.pkl', #'holdout_red2.pkl']):
#    # Load reduced dataset
#    with open(data_path+i, 'rb') as fin:
#        dataset = pkl.load(fin)
#    # Reduce number of families
#    dataset = dataset[dataset['Family ID'].isin(list(range(FAMILIES)))]
#    # Store reduced data
#    with open(data_path+o, 'wb') as fout:
#        pkl.dump(dataset, fout)
#    del dataset

# Use the reduced dataset
with open(data_path+'train_red2.pkl', 'rb') as f:
    train_data = pkl.load(f)
with open(data_path+'valid_red2.pkl', 'rb') as f:
    valid_data = pkl.load(f)

Running the above code reduces the overall data size to around 50 MB training data, 2.5 MB validation data, and unfortunately no holdout data. We shall now plot some characteristics of the training data:

In [ ]:
# Plot the number of proteins in each clan
pdf = pd.DataFrame.from_dict({'Clan ID' : list(range(CLANS)),'Count' : [len(train_data[train_data['Clan ID']==i]) for i in range(CLANS)]})
plt.figure(figsize=(7,4))
sns.barplot(x='Clan ID', y='Count', data=pdf)
plt.tight_layout()
plt.show()

In [ ]:
# Plot the number of families in each clan
pdf = pd.DataFrame.from_dict({'Clan ID' : list(range(CLANS)),'Count' : [len(set(train_data[train_data['Clan ID']==i]['Family ID'])) for i in range(CLANS)]})
plt.figure(figsize=(7,4))
sns.barplot(x='Clan ID', y='Count', data=pdf)
plt.tight_layout()
plt.show()

In [ ]:
# Plot the number of proteins in each family
pdf = pd.DataFrame.from_dict({'Family ID' : list(range(FAMILIES)),'Count' : [len(train_data[train_data['Family ID']==i]) for i in range(FAMILIES)]})
plt.figure(figsize=(15,4))
sns.barplot(x='Family ID', y='Count', data=pdf)
plt.tight_layout()
plt.show()

In [ ]:
# Print some information about the data
print(train_data.info())
print(train_data.head())

And repeat this for the validation data.

In [ ]:
# Plot the number of proteins in each clan
pdf = pd.DataFrame.from_dict({'Clan ID' : list(range(CLANS)),'Count' : [len(valid_data[valid_data['Clan ID']==i]) for i in range(CLANS)]})
plt.figure(figsize=(7,4))
sns.barplot(x='Clan ID', y='Count', data=pdf)
plt.tight_layout()
plt.show()

In [ ]:
# Plot the number of proteins in each clan
pdf = pd.DataFrame.from_dict({'Clan ID' : list(range(CLANS)),'Count' : [len(valid_data[valid_data['Clan ID']==i]) for i in range(CLANS)]})
plt.figure(figsize=(7,4))
sns.barplot(x='Clan ID', y='Count', data=pdf)
plt.tight_layout()
plt.show()

In [ ]:
# Plot the number of proteins in each family
pdf = pd.DataFrame.from_dict({'Family ID' : list(range(FAMILIES)),'Count' : [len(valid_data[valid_data['Family ID']==i]) for i in range(FAMILIES)]})
plt.figure(figsize=(15,4))
sns.barplot(x='Family ID', y='Count', data=pdf)
plt.tight_layout()
plt.show()

In [ ]:
# Print some information about the data
print(valid_data.info())
print(valid_data.head())

We see that the distribution of clans and families is reasonably even across both datasets (as expected). 

## Data Preparation
Getting the Data ready for utilization. This covers changing the Clan IDs, Family IDs and aminoacid sequences into values from 0 to 139 to prepare them for embedding. We place the vocabulary of aminoacids in the beginning of this encoding as their values then correspond directly to their indices in the vocabulary used. This means that all Clan IDs will have an offset of 30, and Family IDs will have an offset of 40. 

Here, the value zero will be used for padding purposes.

### Input

| Feature   | Interval of Values   |
| ------------- |--------------------------|
| Aminoacid vocabulary (30) | $= 0 \ldots 29 $ |
| Clan ID (10)    | $= 30 \ldots 39 $ |
| Family ID (100)    | $= 40 \ldots 139 $ |


### Output

| Feature   | Interval of Values   |
| ------------- |--------------------------|
| Aminoacid vocabulary (30) | $= 0 \ldots 29 $ |

In [ ]:
# Determine clan and family offsets
offset_Clan = VOCAB_SIZE
offset_Family = VOCAB_SIZE + CLANS

# Map offsets onto clan and family IDs
train_data['Clan ID'] += offset_Clan
train_data['Family ID'] += offset_Family
valid_data['Clan ID'] += offset_Clan
valid_data['Family ID'] += offset_Family

# Print some heads
print(train_data.head())
print(valid_data.head())

### Pad all the Inputs and Outputs
Creating Input data and target values with padding - inspired by [this guide](https://towardsdatascience.com/taming-lstms-variable-sized-mini-batches-and-why-pytorch-is-good-for-your-health-61d35642972e). This will make all inputs and outputs the same size. Currently, there is an input X and an output Y, which both need some padding before they can be used for training of the LSTM and the GRU.

In [ ]:
# Import function
from utils import get_data_input

# Padding the input sequence - Clan ID , Family ID and aminoacid sequence (512 Bytes) with padding if less than 512 long.
X_train = get_data_input(train_data, LENGTH)
X_valid = get_data_input(valid_data, LENGTH)

# Print some heads
print(X_train.head())
print(X_valid.head())

In [ ]:
# Import function
from utils import get_data_output

# Padding the output sequence
Y_train = get_data_output(train_data, LENGTH_OUT)
Y_valid = get_data_output(valid_data, LENGTH_OUT)

# Print some heads
print(Y_train.head())
print(Y_valid.head())

We will use a DataLoader to make training more efficient. Unfortunately, Pickle loads the data as an implicit `object_` type (which Pandas also uses to represent DataFrame entries that are not scalars or similar basic types), which must be converted to a numeric type that PyTorch can work with. This is done with the conversion to a Python list below.

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
EPOCHS = 15
BATCH_SIZE = 50
TRAIN_SIZE = 100 # max len(X_train.values)
VALID_SIZE = 2000  # max len(X_valid.values)

# Transform training data first
d_train = torch.tensor([pd.to_numeric(x[0]) for x in X_train.values[:TRAIN_SIZE]]).to(torch.long)
d_targets = torch.tensor([pd.to_numeric(y[0]) for y in Y_train.values[:TRAIN_SIZE]]).to(torch.long)
train_dataset = TensorDataset(d_train, d_targets)
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = False, num_workers = 1)

# Then transform validation data
v_train = torch.tensor([pd.to_numeric(x[0]) for x in X_valid.values[:VALID_SIZE]]).to(torch.long)
v_targets = torch.tensor([pd.to_numeric(y[0]) for y in Y_valid.values[:VALID_SIZE]]).to(torch.long)
valid_dataset = TensorDataset(v_train, v_targets)
valid_loader = DataLoader(valid_dataset, batch_size = BATCH_SIZE, shuffle = False, num_workers = 1)

## RNN Network for Protein Generation
This section will define the network architecture for the neural network RNN used as the backbone of the ProWave neural network for protein generation. Note that this network serves as a baseline implementation meant to test our ideas, while the final network we desire to implement, is a WaveNet-based model, which differs significantly from the network below.

In the following, we consider the so-called validation set as our test set and simply ignore validating the model within each epoch.

We first define an LSTM-based model - heavily inspired by [this guide](https://towardsdatascience.com/taming-lstms-variable-sized-mini-batches-and-why-pytorch-is-good-for-your-health-61d35642972e).

In [ ]:
from model import ProLSTM, train
# LSTM network
netProLSTM = ProLSTM(batch_size=BATCH_SIZE)
netProLSTM = train(netProLSTM, train_loader, epochs = EPOCHS)

# Save LSTM Model
#with open(ModelPath + '/netProLSTM_30Epochs_v5', 'wb') as f:
#    torch.save(netProLSTM.state_dict(), f)

Next, we define a GRU-based model (essentially the same as the LSTM, but with GRU cells).

In [ ]:
from model import ProGRU, train
# GRU network
netProGRU = ProGRU(batch_size=BATCH_SIZE)
netProGRU = train(netProGRU, train_loader, epochs = EPOCHS)

# Save GRU Model
#with open(ModelPath + '/netProGRU_30Epochs_v5', 'wb') as f:
#    torch.save(netProLSTM.state_dict(), f)

## Transformer for Protein Generation
Next, we will define a Transformer model. We define the model based on the tutorial available directly from PyTorch [here](https://pytorch.org/tutorials/beginner/transformer_tutorial.html). 

We use a the template model from the tutorial with few changes to make it match our other networks in size. Some of the savings arise from simply reducing the input and output vocabulary sizes.

In [ ]:
from model import ProTrans, train
# Transformer network
netTrans = ProTrans(nintoken = 140, nouttoken = 30, ninp = 32, nhid = 64, nlayers = 2, nhead = 2, dropout = 0.2)
netTrans = train(netTrans, train_loader, epochs = EPOCHS)

## Wavenet for Protein Generation aka ProWave
We now continue our exploration by defining a WaveNet model - heavily inspired by [this guide](https://github.com/Dankrushen/Wavenet-PyTorch).

We aim to make this model have approximately the same number of weights as the previous models in order to better compare their performances.

In [ ]:
from model import ProWaveNet, nll_loss
# WaveNet network
netProWave = ProWaveNet(LENGTH_OUT, num_layers = 8, num_hidden = 32)
print(netProWave)
netProWave.criterion = nll_loss
netProWave.optimizer = optim.Adam(netProWave.parameters(), lr=0.00086)
netProWave.scheduler = optim.lr_scheduler.StepLR(netProWave.optimizer, step_size = 10, gamma = 0.5)
netProWave.train(train_loader, num_epochs = EPOCHS, disp_interval = 1)
# Generator
netProGen = Generator(netProWave, train_dataset)